In [4]:
# Import Libraries
import requests # gets the website
import pandas as pd #handling tables
from io import StringIO # converts the table to a string

In [5]:
def get_full_ratings_table(year):
    """ 
        Some seasons only have all the teams on one table and some have two tables.
        This function handles both cases by checking the length of the list of DataFrames
        returned by pd.read_html. If there are two tables, it concatenates them.
        If there is only one table, it uses that one. It also checks if the columns are
        multi-indexed and merges them if necessary. It also checks if the required columns
        are present and handles the case where they are not. It also removes asterisks from
        team names and adds a Season column to the DataFrame. Finally, it returns the cleaned
        DataFrame.
    
        Args:
            year (int): The year for which to fetch the ratings table.
        Returns:
            pd.DataFrame: A DataFrame containing the ratings table for the specified year.
    """

    print(f"Fetching data for {year}...")
    
    url = f"https://www.basketball-reference.com/leagues/NBA_{year}_ratings.html"
    res = requests.get(url)
    
    if res.status_code != 200:
        print(f"Failed to fetch page for {year}")
        return None
    
    try:
       all_dfs = pd.read_html(StringIO(res.text))
    except ValueError:
        print(f"No readable tables found for {year}")
        return None
    
    # Some years may have just 1 table
    if len(all_dfs) >= 2:
        df = pd.concat([all_dfs[0], all_dfs[1]], ignore_index=True)
    else:
        df = all_dfs[0]
# Clean up headers if multi-indexed
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.droplevel(0)

    # Try to select columns safely
    try:
        df = df[['Team', 'W', 'L', 'W/L%', 'DRtg']]
    except KeyError as e:
        print(f" Missing columns for {year}: {e}")
        return None
   
    # To fix the error message: SettingWithCopyWarning being that pands thinks I am modifying a copy of the DataFrame
    # instead of the original DataFrame being that we are trying to modify a slice of a DataFrame.
    # Use .loc[:] to show you're modifying the actual DataFrame
    # df.copy() is used to create a copy of the DataFrame to avoid the SettingWithCopyWarning and ensure that the Dataframe is not a view of another DataFrame.
    df= df.copy()
    df.loc[:,'Team'] = df['Team'].str.replace('*', '', regex=False)
    df.loc[:,'Season'] = year

    return df

    

In [6]:
# Loop over seasons 

all_seasons = []

for year in range(2014, 2026):  # Includes 2025
    df = get_full_ratings_table(year)
    if df is not None:
        all_seasons.append(df)

# Combine all years into one big DataFrame
nba_data = pd.concat(all_seasons, ignore_index=True)

print(nba_data)

Fetching data for 2014...
Fetching data for 2015...
Fetching data for 2016...
Fetching data for 2017...
Fetching data for 2018...
Fetching data for 2019...
Fetching data for 2020...
Fetching data for 2021...
Fetching data for 2022...
Fetching data for 2023...
Fetching data for 2024...
Fetching data for 2025...
                      Team   W   L   W/L%    DRtg  Season
0        San Antonio Spurs  62  20  0.756  102.93    2014
1     Los Angeles Clippers  57  25  0.695  105.64    2014
2    Oklahoma City Thunder  59  23  0.720  104.60    2014
3    Golden State Warriors  51  31  0.622  103.07    2014
4          Houston Rockets  54  28  0.659  107.07    2014
..                     ...  ..  ..    ...     ...     ...
355          Brooklyn Nets  26  56  0.317  115.83    2025
356              Utah Jazz  17  65  0.207  120.61    2025
357   New Orleans Pelicans  21  61  0.256  120.01    2025
358      Charlotte Hornets  19  63  0.232  116.65    2025
359     Washington Wizards  18  64  0.220  119.12 

In [7]:
# Save to CSV
nba_data.to_csv('Data/nba_team_ratings.csv', index=False)